In [1]:
from glob import glob
import pandas as pd
from pprint import pprint
from tabulate import tabulate

In [2]:
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

### Short Table

In [3]:
# Read in entangled results
entangled_paths = glob(f"{DATA_PATH}/Models/InterfaceGAN/Outputs/entangled_typicality/*/summary_stats.csv")
entangled_results = pd.concat([pd.read_csv(path) for path in entangled_paths])
entangled_results = entangled_results[entangled_results.n .isin([1000,2000,3000])]
entangled_results.rename(columns= {'acc':'DINOv2'}, inplace=True)
entangled_results

,n,DINOv2
0,3000,0.952222
0,1000,0.981667
0,2000,0.982500


In [4]:
# Read in disentangled results
disentangled_paths = glob(f"{DATA_PATH}/Models/InterfaceGAN/Outputs/disentangled_typicality/*/*/summary_stats.csv")
disentangled_results = pd.concat([pd.read_csv(path) for path in disentangled_paths])
relevant_attributes = ['disentangled_concat_ex_Sleeve_Length', 'disentangled_concat_ex_Color', 'disentangled_embeddings_concat']
disentangled_results = disentangled_results[disentangled_results.attribute.isin(relevant_attributes)]
disentangled_results = disentangled_results.pivot(index = 'n', columns = 'attribute', values = 'acc')
disentangled_results.reset_index(inplace=True)
disentangled_results.columns.name = None

disentangled_results.rename(columns = {
    'disentangled_embeddings_concat':'All', 
    'disentangled_concat_ex_Color':'Excl. Color',
    'disentangled_concat_ex_Sleeve_Length':'Excl. Sleeve Length'
}, inplace=True)
disentangled_results

,n,Excl. Color,Excl. Sleeve Length,All
0,1000,0.996667,0.973333,1.000000
1,2000,0.996667,0.964167,0.993333
2,3000,0.997778,0.939444,0.996667


In [5]:
# Merge them together
results_short = pd.merge(entangled_results, disentangled_results).sort_values('n', ascending = True)
results_short = results_short[['n', 'DINOv2', 'All','Excl. Color','Excl. Sleeve Length']]
results_short = results_short.round(4)

In [6]:
latex_table = tabulate(results_short, headers='keys', tablefmt='latex', showindex=False)
print(latex_table)

\begin{tabular}{rrrrr}
\hline
    n &   DINOv2 &    All &   Excl. Color &   Excl. Sleeve Length \\
\hline
 1000 &   0.9817 & 1      &        0.9967 &                0.9733 \\
 2000 &   0.9825 & 0.9933 &        0.9967 &                0.9642 \\
 3000 &   0.9522 & 0.9967 &        0.9978 &                0.9394 \\
\hline
\end{tabular}


### Complete Tables

In [7]:
# Read in entangled results
conditional_paths = glob(f"{DATA_PATH}/Models/InterfaceGAN/Outputs/e4e_00005_concatenated/*/*/summary_stats.csv")
conditional_results = pd.concat([pd.read_csv(path) for path in conditional_paths])
conditional_results = conditional_results[conditional_results.attribute.isin(['color', 'sleeve_length'])]
conditional_results.rename(columns= {'neg_class':'Attribute 1', 'pos_class':'Attribute 2', 'acc':'Validation Accuracy'}, inplace=True)
conditional_results

,attribute,Attribute 1,Attribute 2,Validation Accuracy
0,sleeve_length,Strapless,Extra long,0.995475
0,sleeve_length,Elbow length,Extra long,0.987147
0,sleeve_length,Elbow length,"Sleeveless, strapless",0.996032
0,sleeve_length,Sleeveless,Strapless,0.962919
0,sleeve_length,Mixed,"Sleeveless, strapless",0.764706
...,...,...,...,...
0,color,beige,green,0.931579
0,color,beige,yellow,0.950845
0,color,brown,beige,0.902466
0,color,beige,black,0.987997


In [28]:
summary_stats = conditional_results.groupby('attribute')['Validation Accuracy'].describe().T
summary_stats.columns.name = None
summary_stats.rename(columns = {'color':'Color', 'sleeve_length':'Sleeve Length'}, inplace=True)
summary_stats = summary_stats.round(4)
summary_stats

,Color,Sleeve Length
count,91.0000,54.0000
mean,0.9595,0.9673
std,0.0324,0.0506
min,0.8736,0.7647
25%,0.9359,0.9650
50%,0.9694,0.9885
75%,0.9859,0.9962
max,1.0000,1.0000


In [29]:
latex_table = tabulate(summary_stats, headers='keys', tablefmt='latex', showindex=True)
print(latex_table)

\begin{tabular}{lrr}
\hline
       &   Color &   Sleeve Length \\
\hline
 count & 91      &         54      \\
 mean  &  0.9595 &          0.9673 \\
 std   &  0.0324 &          0.0506 \\
 min   &  0.8736 &          0.7647 \\
 25\%   &  0.9359 &          0.965  \\
 50\%   &  0.9694 &          0.9885 \\
 75\%   &  0.9859 &          0.9962 \\
 max   &  1      &          1      \\
\hline
\end{tabular}
